In [9]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [18]:
# from huggingface_hub import login
# access_token = ""
# login(access_token)

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
llama_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [14]:
def get_completion_llama(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from meta-llama/Llama-3.2-1B-Instruct:"
    response = ""
    llama_response = llama_pipeline(prompt,
                          max_length=500,
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                          **kwargs,
                       )
    response = llama_response[0]['generated_text']
    return response

### Zero-shot prompting

In [15]:
discrete_prompt = "What is the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1"
response_discrete = get_completion_llama(discrete_prompt)
print(response_discrete)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
What is the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1

#### Response from meta-llama/Llama-3.2-1B-Instruct: 
To calculate the sum of odd numbers from the list, I'll first identify the odd numbers in the list. The odd numbers in the list are 15, 13, 5, 1.

#### Response from meta-llama/Llama-3.2-1B-Instruct: 
I'll calculate the sum of the odd numbers. The sum of 15, 13, 5, and 1 is 34.

#### Response from meta-llama/Llama-3.2-1B-Instruct: 
The sum of the odd numbers from the list is 34.


### Few-shot prompting

In [17]:
few_shot_prompt = """Follow the examples below that the sum of odd numbers in a list of given integers
Example 1:
List: 4, 8, 9, 15, 12, 2, 1
Sum: 25

Example 2:
List: 17, 10, 19, 4, 8, 12, 24
Sum: 36

Example 3:
List: 16, 11, 14, 4, 8, 13, 24
Sum: 24

Now, what is the sum of odd numbers from the list: 15, 32, 5, 13, 82, 7, 1
"""

response_fshot = get_completion_llama(few_shot_prompt)
print(response_fshot)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
Follow the examples below that the sum of odd numbers in a list of given integers
Example 1:
List: 4, 8, 9, 15, 12, 2, 1
Sum: 25

Example 2:
List: 17, 10, 19, 4, 8, 12, 24
Sum: 36

Example 3:
List: 16, 11, 14, 4, 8, 13, 24
Sum: 24

Now, what is the sum of odd numbers from the list: 15, 32, 5, 13, 82, 7, 1


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
You want to find the sum of odd numbers from a list. To do this, you can use a simple loop and check each number in the list to see if it is odd. If it is, you add it to a running total. Here is a Python function that does this:

```python
def sum_odd_numbers(numbers):
    total = 0
    for num in numbers:
        if num % 2!= 0:  # Check if the number is odd
            total += num
    return total

numbers = [15, 32, 5, 13, 82, 7, 1]
print(sum_odd_numbers(numbers))
```

This function works by iterating over each number in the list, and adding it to a running total if it is odd. When the loop finishes, the function 

### Chain of thought